In [124]:
import tensorflow as tf
import pandas as pd
import os
import sys
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model
from sklearn.model_selection import train_test_split

In [125]:
def word_split(mystring):
    str_split = []
    for tmp in mystring:
        tmp = re.sub('[^a-zA-Z0-9\s]+', '', tmp)
        tmp = tmp.split(' ')
        while True:
            if '' not in tmp:
                break
            tmp.remove('')    
        str_split.append(tmp)
    return str_split

In [132]:
GLOVE_DIR = './'
MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 100
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [127]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [128]:
all_data = pd.read_csv("train.csv")
data = all_data['Headline']
label = all_data['Label']
my_split = word_split(data)

In [129]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(my_split)
sequences = tokenizer.texts_to_sequences(my_split)
word_index = tokenizer.word_index
x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [130]:
x_train, x_test, y_train, y_test = train_test_split(x, label, test_size=0.1, random_state = 42)

In [133]:
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i-1] = embedding_vector

In [134]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [135]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
NUM_LSTM_UNITS = 32
shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(embedded_sequences)
dense = Dense(units=1, activation='linear')
predictions = dense(top_output)
model = Model(inputs=sequence_input, outputs=predictions)

In [136]:
model.compile(loss='MSE',
              optimizer='Adam',)
model.summary()

Model: "functional_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 100, 100)          10000     
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                17024     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 27,057
Trainable params: 17,057
Non-trainable params: 10,000
_________________________________________________________________


In [137]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
15/15 [==============================] - 1s 48ms/step - loss: 5.5196 - val_loss: 1.0232
Epoch 2/10
15/15 [==============================] - 0s 30ms/step - loss: 0.6329 - val_loss: 0.7162
Epoch 3/10
15/15 [==============================] - 0s 30ms/step - loss: 0.5760 - val_loss: 0.6044
Epoch 4/10
15/15 [==============================] - 0s 30ms/step - loss: 0.5295 - val_loss: 0.5980
Epoch 5/10
15/15 [==============================] - 0s 30ms/step - loss: 0.5202 - val_loss: 0.5890
Epoch 6/10
15/15 [==============================] - 0s 30ms/step - loss: 0.5124 - val_loss: 0.5871
Epoch 7/10
15/15 [==============================] - 0s 30ms/step - loss: 0.5078 - val_loss: 0.5820
Epoch 8/10
15/15 [==============================] - 0s 29ms/step - loss: 0.5061 - val_loss: 0.5827
Epoch 9/10
15/15 [==============================] - 0s 30ms/step - loss: 0.5026 - val_loss: 0.5795
Epoch 10/10
15/15 [==============================] - 0s 30ms/step - loss: 0.5001 - val_loss: 0.5812


In [142]:
test_data = pd.read_csv("test.csv")
data = test_data['Headline']
label = test_data['Label']
test_split = word_split(data)
y_pre = model.predict(x_test)
b = np.arange(1, y_pre.shape[0]+1).reshape(y_pre.shape[0], 1)
y_pre = np.append(b, y_pre, axis=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID','Label'])
    writer.writerows(y_pre)